In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

In [ ]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
    }
)
Pacientes_SF.shape



In [ ]:
results = sf.query_all(
    """
    select id,IdentificationId__c from Contact 
   """
)
Pacientes_SF2 = pd.DataFrame(results["records"])

Pacientes_SF2 = Pacientes_SF2[
    [
        "Id",
        "IdentificationId__c",
    ]
]
Pacientes_SF2 = Pacientes_SF2.rename(
    columns={
        "Id": "Id Contact Paciente",
        "IdentificationId__c": "Rut",
    }
)
Pacientes_SF2.shape

In [ ]:
Pacientes_SF2 = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_SF2,
    how="left",
    left_on="Rut",
    right_on="Rut",
)
Pacientes_SF2.shape

In [12]:
results = sf.query_all(
    """
    select id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
   """
)
Paciente_Programa = pd.DataFrame(results["records"])

Paciente_Programa = Paciente_Programa[
    [
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Paciente_Programa = Paciente_Programa.rename(
    columns={
        "Paciente__c": "Id paciente account",
        "Programas_por_Empresas__c": "Programas por empresa",
    }
)
Paciente_Programa.shape

(1357715, 2)

In [13]:

nuevos = "F:/Empresas SF/reportes/empresas.xlsx"
empresas = pd.read_excel(nuevos)
empresas = empresas[
    [
        "Programas_por_Empresas__c",
        "Id Account Empresa",
        
    ]
]
empresas.head()


,Programas_por_Empresas__c,Id Account Empresa
0,NaN,0018c00002mTJMAAA4
1,a1W8c000008TC04EAG,0018c00002mUFraAAG
2,a1W8c0000083oJOEAY,0018c00002f27paAAA
3,a1W8c0000083qXGEAY,0018c00002ghxCCAAY
4,a1W8c0000083jGVEAY,0018c00002cY37kAAC


In [16]:
Paciente_Programa = pd.merge(
    left=Paciente_Programa,
    right=empresas,
    how="left",
    left_on="Programas por empresa",
    right_on="Programas_por_Empresas__c",
)
Paciente_Programa.shape

(1357715, 4)

In [17]:
results = sf.query_all(
    """
    select AccountId, ContactId from AccountContactRelation 
   """
)
accountcontact = pd.DataFrame(results["records"])

accountcontact = accountcontact[
    [
        "AccountId",
        "ContactId",
    ]
]
accountcontact = accountcontact.rename(
    columns={
        "AccountId": "Id account",
        "ContactId": "id contacto",
    }
)
accountcontact.shape

(1357715, 4)

In [19]:
accountcontact['LLave'] = accountcontact['id contacto'] + accountcontact['Id account'] 
accountcontact.head()

,Id account,id contacto,LLave
0,0018c00002axQdlAAE,0038c000036jelbAAA,0038c000036jelbAAA0018c00002axQdlAAE
1,0018c00002axQdlAAE,0038c000035CWLYAA4,0038c000035CWLYAA40018c00002axQdlAAE
2,0018c00002axQdlAAE,0038c000035CaxiAAC,0038c000035CaxiAAC0018c00002axQdlAAE
3,0018c00002axQdlAAE,0038c000036jelcAAA,0038c000036jelcAAA0018c00002axQdlAAE
4,0018c00002axQa5AAE,0038c000036jLDoAAM,0038c000036jLDoAAM0018c00002axQa5AAE


In [20]:
Paciente_Programa.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM,a1W8c000007vDPvEAM,0018c00002axQaeAAE
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE


In [21]:
Pacientes_SF2.head()

,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002amLTMAA2,173805004,0038c000035CV93AAG
1,0018c00002amLUGAA2,153338949,0038c000035CV94AAG
2,0018c00002amLUHAA2,177735213,0038c000035CV95AAG
3,0018c00002amLUIAA2,169332924,0038c000035CV9zAAG
4,0018c00002amLUJAA2,172957692,0038c000035CVA0AAO


In [22]:
programa_paciente = pd.merge(
    left=Paciente_Programa,
    right=Pacientes_SF2,
    how="left",
    left_on="Id paciente account",
    right_on="Id Account Paciente",
)
programa_paciente.shape

(1357724, 7)

In [23]:
programa_paciente.head()

,Id paciente account,Programas por empresa,Programas_por_Empresas__c,Id Account Empresa,Id Account Paciente,Rut,Id Contact Paciente
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM,a1W8c000007vDPvEAM,0018c00002axQaeAAE,0018c00002amLbpAAE,224488556,0038c000035CVHVAA4
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLbqAAE,113376066,0038c000035CVHWAA4
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLbrAAE,79476161,0038c000035CVHXAA4
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLbsAAE,181854227,0038c000035CVHYAA4
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,0018c00002axQecAAE,0018c00002amLc2AAE,126899823,0038c000035CVHnAAO


In [25]:
programa_paciente = programa_paciente[
    [
        "Id Contact Paciente",
        "Id Account Empresa",
    ]
]
programa_paciente['LLave'] = programa_paciente['Id Contact Paciente'] + programa_paciente['Id Account Empresa']
programa_paciente.head()

,Id Contact Paciente,Id Account Empresa,LLave
0,0038c000035CVHVAA4,0018c00002axQaeAAE,0038c000035CVHVAA40018c00002axQaeAAE
1,0038c000035CVHWAA4,0018c00002axQecAAE,0038c000035CVHWAA40018c00002axQecAAE
2,0038c000035CVHXAA4,0018c00002axQecAAE,0038c000035CVHXAA40018c00002axQecAAE
3,0038c000035CVHYAA4,0018c00002axQecAAE,0038c000035CVHYAA40018c00002axQecAAE
4,0038c000035CVHnAAO,0018c00002axQecAAE,0038c000035CVHnAAO0018c00002axQecAAE


In [26]:
programa_paciente["RutEstaoNo"] = programa_paciente["LLave"].isin(accountcontact['LLave'])

In [27]:
programa_paciente.head()

,Id Contact Paciente,Id Account Empresa,LLave,RutEstaoNo
0,0038c000035CVHVAA4,0018c00002axQaeAAE,0038c000035CVHVAA40018c00002axQaeAAE,True
1,0038c000035CVHWAA4,0018c00002axQecAAE,0038c000035CVHWAA40018c00002axQecAAE,True
2,0038c000035CVHXAA4,0018c00002axQecAAE,0038c000035CVHXAA40018c00002axQecAAE,True
3,0038c000035CVHYAA4,0018c00002axQecAAE,0038c000035CVHYAA40018c00002axQecAAE,True
4,0038c000035CVHnAAO,0018c00002axQecAAE,0038c000035CVHnAAO0018c00002axQecAAE,True


In [28]:
programa_paciente = programa_paciente[programa_paciente["RutEstaoNo"] == False]
programa_paciente.head()

,Id Contact Paciente,Id Account Empresa,LLave,RutEstaoNo
385,0038c000035CWNUAA4,0018c00002cXJkEAAW,0038c000035CWNUAA40018c00002cXJkEAAW,False
461,0038c000035CWQdAAO,0018c00002gi3QkAAI,0038c000035CWQdAAO0018c00002gi3QkAAI,False
1083,0038c000035CXQiAAO,0018c00002gi3TzAAI,0038c000035CXQiAAO0018c00002gi3TzAAI,False
1317,0038c000035CXEYAA4,0018c00002gi3U2AAI,0038c000035CXEYAA40018c00002gi3U2AAI,False
2302,0038c000035CZxVAAW,0018c00002cXcjYAAS,0038c000035CZxVAAW0018c00002cXcjYAAS,False


In [29]:
programa_paciente.shape

(487661, 4)

In [30]:
programa_paciente.to_excel("F:/Empresas SF/reportes/accountcontact.xlsx", index=False)